In [ ]:
import chess
import numpy

squares_index = {
    'a': 0,
    'b': 1,
    'c': 2,
    'd': 3,
    'e': 4,
    'f': 5,
    'g': 6,
    'h': 7
}


# example: h3 -> 17
def square_to_index(square):
  letter = chess.square_name(square)
  return 8 - int(letter[1]), squares_index[letter[0]]


def split_dims(board):
  # this is the 3d matrix
  board3d = numpy.zeros((14, 8, 8), dtype=numpy.int8)

  # here we add the pieces's view on the matrix
  for piece in chess.PIECE_TYPES:
    for square in board.pieces(piece, chess.WHITE):
      idx = numpy.unravel_index(square, (8, 8))
      board3d[piece - 1][7 - idx[0]][idx[1]] = 1
    for square in board.pieces(piece, chess.BLACK):
      idx = numpy.unravel_index(square, (8, 8))
      board3d[piece + 5][7 - idx[0]][idx[1]] = 1

  # add attacks and valid moves too
  # so the network knows what is being attacked
  aux = board.turn
  board.turn = chess.WHITE
  for move in board.legal_moves:
      i, j = square_to_index(move.to_square)
      board3d[12][i][j] = 1
  board.turn = chess.BLACK
  for move in board.legal_moves:
      i, j = square_to_index(move.to_square)
      board3d[13][i][j] = 1
  board.turn = aux

  return board3d


In [ ]:
import numpy
import tensorflow.keras.callbacks as callbacks


def get_dataset():
	# Download dataset.npz from the link in Readme.md
	container = numpy.load('dataset.npz')
	b, v = container['b'], container['v']
	v = numpy.asarray(v / abs(v).max() / 2 + 0.5,
	                  dtype=numpy.float32)  # normalization (0 - 1)
	return b, v


x_train, y_train = get_dataset()

In [ ]:
# Split dataset for two clients
# pip install -U scikit-learn
from sklearn.model_selection import train_test_split

x_train, y_train, x_test, y_test = train_test_split(
    x_train, y_train, test_size=0.9, random_state=100)
x_train, y_train, x_test, y_test = train_test_split(
    x_train, y_train, test_size=0.3, random_state=100)
fed_x_train, fed_y_train, x_test, y_test = train_test_split(
    x_test, y_test, test_size=0.5, random_state=100)

In [ ]:
import tensorflow.keras.models as models
import tensorflow.keras.layers as layers
import tensorflow.keras.utils as utils
import tensorflow.keras.optimizers as optimizers


def build_model(conv_size, conv_depth):
  board3d = layers.Input(shape=(14, 8, 8))

  # adding the convolutional layers
  x = board3d
  for _ in range(conv_depth):
    x = layers.Conv2D(filters=conv_size, kernel_size=3,
                      padding='same', activation='relu')(x)
  x = layers.Flatten()(x)
  x = layers.Dense(64, 'relu')(x)
  x = layers.Dense(1, 'sigmoid')(x)

  return models.Model(inputs=board3d, outputs=x)

In [ ]:
model = build_model(32, 4)

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
model.compile(optimizer=optimizers.Adam(5e-4), loss='mean_squared_error')
model.summary()

model.fit(x_train, y_train,
          batch_size=2048,
          epochs=10,
          verbose=1,
          validation_split=0.1,
          callbacks=[callbacks.ReduceLROnPlateau(monitor='loss', patience=10),
                     callbacks.EarlyStopping(monitor='loss', patience=15, min_delta=1e-4)])

model.save_weights("model_weight/new_cen_weights", save_format="h5")

In [ ]:
fed1 = model
fed2 = model

In [ ]:
fed1.fit(x_train, y_train,
          batch_size=2048,
          epochs=10,
          verbose=1,
          validation_split=0.1,
          callbacks=[callbacks.ReduceLROnPlateau(monitor='loss', patience=10),
                     callbacks.EarlyStopping(monitor='loss', patience=15, min_delta=1e-4)])


In [ ]:
fed2.fit(x_train, y_train,
          batch_size=2048,
          epochs=10,
          verbose=1,
          validation_split=0.1,
          callbacks=[callbacks.ReduceLROnPlateau(monitor='loss', patience=10),
                     callbacks.EarlyStopping(monitor='loss', patience=15, min_delta=1e-4)])


In [ ]:
fed1_weights = fed1.get_weights()
fed2_weights = fed2.get_weights()

In [ ]:
client1_weights = numpy.array(fed1_weights, dtype=object)
client2_weights = numpy.array(fed2_weights, dtype=object)
aggregated_weights = numpy.add(client1_weights,client2_weights)
fed1.set_weights(aggregated_weights)
fed1.save_weights("model_weight/new_fed_weights", save_format="h5")

In [ ]:
result0 = model.evaluate(x_test, y_test, batch_size = 32)
print("test loss, test acc:", result0)

In [ ]:
result1 = fed1.evaluate(x_test, y_test, batch_size = 32)
print("test loss, test acc:", result1)